### Student Information
Name: 莊孝萱

Student ID: 106062105

GitHub ID: shiuanC

Kaggle name: SharonDieting

---

**This notebook only have the Part 2 and 3**

# Step 0 



In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/MyDrive/Colab Notebooks/lab2"

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/lab2


In [2]:
### Begin Assignment Here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tq

from datetime import datetime

# step 1

## compress size for saving memory space

In [3]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% Compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
def resumetable(df):
    print(f'Shape : {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Feature'})
    summary['Num of null'] = df.isnull().sum().values
    summary['Num of unique'] = df.nunique().values
    summary['First value'] = df.loc[0].values
    summary['Second value'] = df.loc[1].values
    summary['Third value'] = df.loc[2].values
    return summary

## preprocess

### load data

In [6]:
### training data
import json 
## read in the main data set
json_name = 'part2-data/tweets_DM.json'

In [ ]:
file = open(json_name, 'r', encoding='utf-8')
json_list = []
for line in tq.tqdm(file.readlines()):
    dic = json.loads(line)
    json_list.append(dic)
print(len(json_list))

  0%|          | 0/1867535 [00:00<?, ?it/s]

1867535


In [ ]:
print(type(json_list), type(json_list[0]))

<class 'list'> <class 'dict'>


In [ ]:
df = pd.json_normalize(json_list)

In [ ]:
df.head()

,_score,_index,_crawldate,_type,_source.tweet.hashtags,_source.tweet.tweet_id,_source.tweet.text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [ ]:
# since I use google colab to train I have to free the ram space 
del json_list

In [ ]:
emotion = pd.read_csv ('part2-data/emotion.csv')
data_id = pd.read_csv ('part2-data/data_identification.csv')


### other preprocess

drop useless cols
and rename

In [ ]:
df = df.drop(columns=['_crawldate', '_index', '_type','_score'])
df = df.rename(columns={"_source.tweet.text": "text",
                        "_source.tweet.tweet_id": "tweet_id",
                        "_source.tweet.hashtags": "hashtags"})
df.head()

,hashtags,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [ ]:
# merge all data
emo_idf=pd.merge(data_id,emotion,on='tweet_id',how='left')
df= pd.merge(df,emo_idf, on='tweet_id',how='left')

**append hashtag to text ** <br>
As a trial I add the hash tag <br>
sort of adding the weight of hash tag <br>
(however, just totally don't work


In [ ]:
df['hashtags'] = [' '.join(map(str, l)) for l in df['hashtags']]
type(df.hashtags[1])

str

In [ ]:
df.hashtags[1]

'freepress TrumpLegacy CNN'

In [ ]:
#append hashtag to the end of text
df['textNtags'] = df['text'].astype(str) + ' '+ df['hashtags'].astype(str)
df.text[1]

'@brianklaas As we see, Trump is dangerous to #freepress around the world. What a <LH> <LH> #TrumpLegacy.  #CNN'

In [ ]:
df.textNtags[1]

'@brianklaas As we see, Trump is dangerous to #freepress around the world. What a <LH> <LH> #TrumpLegacy.  #CNN freepress TrumpLegacy CNN'

In [ ]:
# use tweet_id as index
df = df.set_index('tweet_id')

In [ ]:
df.head()

,hashtags,text,identification,emotion,textNtags
tweet_id,,,,,
0x376b20,Snapchat,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,"People who post ""add me on #Snapchat"" must be ..."
0x2d5350,freepress TrumpLegacy CNN,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,"@brianklaas As we see, Trump is dangerous to #..."
0x28b412,bibleverse,"Confident of your obedience, I write to you, k...",test,NaN,"Confident of your obedience, I write to you, k..."
0x1cd5b0,,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,Now ISSA is stalking Tasha 😂😂😂 <LH>
0x2de201,,"""Trust is not the same as faith. A friend is s...",test,NaN,"""Trust is not the same as faith. A friend is s..."


## preprocess conti

In [ ]:
import re
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

## split data

In [ ]:
train_df = df.loc[df['identification'] == 'train'].drop(columns=['identification'])
test_df = df.loc[df['identification'] == 'test'].drop(columns=['emotion', 'identification'])
train_df.head()

,tweet_id,hashtags,text,emotion,textNtags
0,0x376b20,Snapchat,"People who post ""add me on #Snapchat"" must be ...",anticipation,"People who post ""add me on #Snapchat"" must be ..."
1,0x2d5350,freepress TrumpLegacy CNN,"@brianklaas As we see, Trump is dangerous to #...",sadness,"@brianklaas As we see, Trump is dangerous to #..."
3,0x1cd5b0,NaN,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear,Now ISSA is stalking Tasha 😂😂😂 <LH>
5,0x1d755c,authentic LaughOutLoud,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy,@RISKshow @TheKevinAllison Thx for the BEST TI...
6,0x2c91a8,NaN,Still waiting on those supplies Liscus. <LH>,anticipation,Still waiting on those supplies Liscus. <LH>


In [ ]:
train_df.emotion.unique()

array(['anticipation', 'sadness', 'fear', 'joy', 'anger', 'trust',
       'disgust', 'surprise'], dtype=object)

In [ ]:
emotion_map ={
    'anticipation':0,
    'sadness':1,
    'fear':2,
    'joy':3,
    'trust':4,
    'disgust':5,
    'surprise':6,
    'anger':7
}

In [ ]:
train_df['emotion_id'] = train_df.emotion.map(emotion_map)
train_df.head()

,tweet_id,hashtags,text,emotion,textNtags,emotion_id
0,0x376b20,Snapchat,"People who post ""add me on #Snapchat"" must be ...",anticipation,"People who post ""add me on #Snapchat"" must be ...",0
1,0x2d5350,freepress TrumpLegacy CNN,"@brianklaas As we see, Trump is dangerous to #...",sadness,"@brianklaas As we see, Trump is dangerous to #...",1
3,0x1cd5b0,NaN,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear,Now ISSA is stalking Tasha 😂😂😂 <LH>,2
5,0x1d755c,authentic LaughOutLoud,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy,@RISKshow @TheKevinAllison Thx for the BEST TI...,3
6,0x2c91a8,NaN,Still waiting on those supplies Liscus. <LH>,anticipation,Still waiting on those supplies Liscus. <LH>,0


In [ ]:
train_df = train_df.set_index('tweet_id')
test_df = test_df.set_index('tweet_id')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1455563 entries, 0x376b20 to 0x34be8c
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   hashtags    759202 non-null   object
 1   text        1455563 non-null  object
 2   emotion     1455563 non-null  object
 3   textNtags   1455563 non-null  object
 4   emotion_id  1455563 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 66.6+ MB


In [ ]:
tmp = train_df.text[0]

In [ ]:
text_preprocessing(tmp)

'people post add snapchat must dehydrated cuz man lh'

# Step 2 train model trials

## the real submission method: tfidf+NB
Because in the group project, our group eventually only two people is doing project. As the only bachelor student in this group, somehow I become responsible the main part of the group project. I almost devote over 80% of the main part. <Br>
I JUST REALLY ANGRY!!!! <Br>
cause I really only don't have time to finish my own hw <br>
so at the end i simply use the method taught in part 1

### load the data

In [ ]:
X_test = test_df['text']
X_test_id = test_df.index
X_preprocessed = np.array([text_preprocessing(text) for text in tqdm.tqdm(X_test)])


make the tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Calculate TF-IDF

tf_idf = TfidfVectorizer(ngram_range=(1, 3), 
                         max_features=3000,
                         stop_words=stopwords.words('english'),
                         smooth_idf=False)

tf_idf.fit(X_preprocessed)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Calculate TF-IDF

tf_idf = TfidfVectorizer(max_features=3000, stop_words=stopwords.words('english')) 
tf_idf.fit(X_preprocessed)

In [ ]:
X_tfidf = tf_idf.transform(X_preprocessed)


In [ ]:
from sklearn.model_selection import train_test_split

X_train_tfidf, X_val_tfidf, y_train, y_val =\
    train_test_split(X_tfidf, y, test_size=0.2, random_state=2030)

### naive bayse

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from sklearn.naive_bayes import MultinomialNB


model_tf_NB = MultinomialNB()
model_tf_NB.fit(X_tfidf, y)
model_tf_NB.score(X_val_tfidf, y_val)

In [ ]:
res_tfidf_NB = model_tf_NB.predict(X_test_tfidf)
 

In [ ]:
out_tfidf_NB = pd.DataFrame({'emoId':res_tfidf_NB}, columns = ['emoId'],index=X_test_id)
out_tfidf_NB.index.names = ['id']
out_tfidf_NB['emotion'] = out_tfidf_NB.emoId.map(re_emotion_map)
out_tfidf_NB = out_tfidf_NB.drop(columns = 'emoId')
out_tfidf_NB.head()

In [ ]:
out_tfidf_NB.to_csv('output/tfidf_NB.csv')


## other attemption1: roberta

After doing some research I fount the roberta model perform extremely good in most sentiment analysis completion<br>


---

WELL, i end up give up this method as really require the good GPU and lots of ram



In [ ]:
!pip install transformers==3.4.0

     |████████████████████████████████| 1.3 MB 4.6 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 1.2 MB 21.1 MB/s 
     |████████████████████████████████| 2.9 MB 41.4 MB/s 


In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

the below cell is the trail to use colab's tpu

In [ ]:
# imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
train_df.shape

(1455563, 5)

In [ ]:
new_df = train_df[['text', 'emotion_id']]

### Preparing the Dataset and Dataloader

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.emotion_id
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
extract_df = new_df.sample(frac=0.1, replace=True, random_state=1999)
extract_df.shape

(145556, 2)

In [ ]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=30)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(extract_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
# testing_set = SentimentData(test_data, tokenizer, MAX_LEN)
testing_set = SentimentData(extract_df, tokenizer, MAX_LEN)

FULL Dataset: (145556, 2)
TRAIN Dataset: (1164450, 2)
TEST Dataset: (291113, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### Creating the Neural Network for Fine Tuning

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 8)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

### Fine Tuning the Model

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    # RESUME = True
    RESUME = False

    if RESUME:
        path_checkpoint='test_0_37.60683760683761.pth'
        checkpoint = torch.load(path_checkpoint, map_location=torch.device(device))
        model.load_state_dict(checkpoint)
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            print(_)
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"\n Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps

    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    now = datetime.now().strftime("%H%M")
    torch.save(model.state_dict(), 'test_{}_{}.pth'.format(epoch,now))

    return 

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



 Training Loss per 5000 steps: 2.0730648040771484
Training Accuracy per 5000 steps: 25.0


728it [09:44,  1.25it/s]


The Total Accuracy for Epoch 0: 42.69643623872907
Training Loss Epoch: 1.5820456637622236
Training Accuracy Epoch: 42.69643623872907


0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 8: 90.27908973808502
Training Loss Epoch: 0.30685247925550235
Training Accuracy Epoch: 90.27908973808502


0it [00:00, ?it/s]


 Training Loss per 5000 steps: 0.20299670100212097
Training Accuracy per 5000 steps: 93.75


728it [09:38,  1.26it/s]


The Total Accuracy for Epoch 9: 92.35723486474882
Training Loss Epoch: 0.24306369944383482
Training Accuracy Epoch: 92.35723486474882


In [ ]:
path_checkpoint='test_8_0101_1233.pth'
checkpoint = torch.load(path_checkpoint, map_location=torch.device(device))
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0 & _!=0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

0it [00:00, ?it/s]Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
2it [00:00,  3.79it/s]

Validation Loss per 100 steps: 2.2544167041778564
Validation Accuracy per 100 steps: 25.0


2541it [07:54,  5.35it/s]


KeyboardInterrupt: ignored